In [1]:
with open('input.txt', 'r', encoding='utf-8') as f:
      text= f.read()

In [2]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [3]:
stoi = {ch : i for i,ch in enumerate(chars)}
itos = {i : ch for i,ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("om kumar"))
print(decode(encode("om kumar")))

# we can use tiktoken library for its encoder and decoder which gpt uses itself, here we're keeping it simple 

[53, 51, 1, 49, 59, 51, 39, 56]
om kumar


In [4]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [5]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [6]:
block_size = 8
len(train_data), train_data[:block_size]

(1003854, tensor([18, 47, 56, 57, 58,  1, 15, 47]))

In [7]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
      data = train_data if split == 'train' else val_data
      ix = torch.randint(len(data) - block_size, (batch_size,))
      x = torch.stack([data[i:i+block_size] for i in ix])
      y = torch.stack([data[i+1:i+block_size+1] for i in ix])
      return x,y

xb, yb = get_batch('train')
print('input: ',xb.shape)
print(xb)
print('targets: ',yb.shape)
print(yb)

print('-------------')

# for b in range(batch_size):
      # for t in range(block_size):
            # context = xb[b, :t+1]
            # target = yb[b,t]
            # print(f'when input is {context.tolist()} the target is {target}')
      

input:  torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:  torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-------------


In [8]:
import torch
import torch.nn as nn 
from torch.nn import functional as F 
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
      
      def __init__(self, vocab_size):
            super().__init__()
            # each token directly reads off the logits for the next token from a lookup table
            self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
      
      def forward(self, idx, targets=None):
            # idx and targets are both (B,T) tensor of integers
            logits = self.token_embedding_table(idx) # (B,T,C)
            
            if targets is None:
                  loss = None
            else:
                  B,T,C = logits.shape
                  logits = logits.view(B*T, C)
                  targets = targets.view(B*T)
                  loss = F.cross_entropy(logits, targets)
            
            return logits, loss
      
      def generate(self, idx, max_new_tokens):
            # idx is (B,T) array of indices in the current context
            for _ in range(max_new_tokens):
                  # get the predictions
                  logits,loss = self(idx)
                  # focus only on the last time step
                  logits = logits[:, -1,:] # becomes (B,C) # (B,T,C) ke T dim ke last C ko pick kra
                  # apply softmax to get probabilities
                  probs = F.softmax(logits, dim=1) # (B,C)
                  # sample from the distribution
                  idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
                  # append sampled index to the running sequence
                  idx = torch.cat((idx, idx_next), dim=1) # (B,T+1)
            return idx
      
      
      
      
      
      
      


In [9]:
m = BigramLanguageModel(vocab_size)
logits,loss = m(xb,yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [10]:
print( decode(m.generate(idx= torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [11]:
# create a pytorch optimizer

optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [12]:
batch_size = 32

for steps in range(10001):
      xb,yb = get_batch('train')
      
      logits,loss = m(xb,yb)
      optimizer.zero_grad(set_to_none=True)
      loss.backward()
      optimizer.step()
      
      if(steps%500==0):
            print(loss.item()) 

4.704006195068359
4.241008758544922
3.7031264305114746
3.4233598709106445
3.1371781826019287
2.9196817874908447
2.776794672012329
2.809856653213501
2.5844571590423584
2.6057393550872803
2.5105180740356445
2.5480153560638428
2.531585931777954
2.4708240032196045
2.504757881164551
2.400172472000122
2.4696712493896484
2.4209394454956055
2.4838879108428955
2.407996654510498
2.4183998107910156


In [13]:
print( decode(m.generate(idx= torch.zeros((1,1), dtype=torch.long), max_new_tokens=400)[0].tolist()))


Thicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y heltieiengerofo'dsssit ey
KIN d pe wither vouprrouthercc.
hathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so it t jod weancotha:
h hay.JUCle n prids, r loncave w hollular s O:
HIs; ht anjx?

DUThinqunt.

LaZAnde.
athave l.
KEONH:
ARThanco be y,-hedarwnoddy scace, tridesar, wnl'shenous s ls, theresseys
Plorseelaping


In [14]:
#### SELF ATTENTION

In [15]:
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time channel

x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [16]:
# we want x[b,t] = mean_{i<=t} x[b,i]

xbow = torch.zeros((B,T,C)) # x bag of words

for b in range(B):
      for t in range(T):
            xprev = x[b,:t+1] # (t,C)
            xbow[b,t] = torch.mean(xprev,0)
            
# we can replicate above process using matrix multiplication as done below, u should see explanation sir gave

# better way to do above operation

wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1,keepdim=True)

xbow2 = wei @ x # (T,T)@(B,T,C) ---> (B,T,T)@(B,T,C) ----> (B,T,C)
       

In [17]:
#another way of doing above trick

tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T)) # we dont want uniform initialisation, so to learn from the past, we introduce key and query
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim =-1)
xbow3 = wei @ x

In [18]:
n_embd = 32

In [24]:
import torch
import torch.nn as nn 
from torch.nn import functional as F 
torch.manual_seed(1337)

class Head(nn.Module):
      """ one head of self attention """
      
      def __init__(self, head_size):
            super().__init__()
            self.key = nn.Linear(n_embd, head_size, bias= False)
            self.query = nn.Linear(n_embd, head_size, bias=False)
            self.value = nn.Linear(n_embd, head_size, bias=False)
            self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))
      
      def forward(self, x):
            B,T,C = x.shape
            k = self.key(x)
            q = self.key(x)
            # compute attention scores ("affinities")
            wei = q @ k.transpose(-2,-1) * C**-0.5 # (B,T,16) @ (B,16,T) --> (B,T,T)
            wei = wei.masked_fill(self.tril[:T,:T]==0, float('-inf')) #(B,T,T)
            wei = F.softmax(wei, dim=-1) # (B,T,T)
            # perform the weighed aggregation of the values
            v = self.value(x)
            out = wei @ v # (B,T,T) @ (B,T,C) --> (B,T,C)
            
            return out
      

class MultiHeadAttention(nn.Module):
      """ multiple heads of self attention in parallel """
      
      def __init__(self, num_heads, head_size):
            super().__init__()
            self.heads = nn.ModuleList((Head(head_size) for _ in range(num_heads)))
            self.proj = nn.Linear(n_embd, n_embd)
      
      def forward(self, x):
            out = torch.cat([h(x) for h in self.heads], dim=-1)
            out = self.proj(out)
            return out

class FeedForward(nn.Module):
      """ a simple linear layer followed by a non linearity """
      
      def __init__(self, n_embd):
            super().__init__()
            self.net = nn.Sequential(
                  nn.Linear(n_embd, 4*n_embd),
                  nn.ReLU(),
                  nn.Linear(4*n_embd, n_embd),
            )
      
      def forward(self,x):
            return self.net(x)


class Block(nn.Module):
      """ Transformer block: communication followed by computation """
      
      def __init__(self, n_embd, n_head):
            # n_embd: embedding dimension, n_head: the number of heads we'd like
            super().__init__()
            head_size = n_embd // n_head
            self.sa = MultiHeadAttention(n_head, head_size)
            self.ffwd = FeedForward(n_embd)
            self.ln1 = nn.LayerNorm(n_embd)
            self.ln2 = nn.LayerNorm(n_embd)
            
      
      def forward(self,x):
            x = x+ self.sa(self.ln1(x))
            x = x+ self.ffwd(self.ln2(x))
            return x




class BigramLanguageModel(nn.Module):
      
      def __init__(self):
            super().__init__()
            # each token directly reads off the logits for the next token from a lookup table
            self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
            self.position_embedding_table = nn.Embedding(block_size, n_embd)
            # self.sa_head = Head(n_embd)
            # self.sa_heads = MultiHeadAttention(4, n_embd//4) # i.e. 4 heads of 8 dimensional self attention
            # self.ffwd = FeedForward(n_embd)
            self.blocks = nn.Sequential(
                  Block(n_embd, n_head=4),
                  Block(n_embd, n_head=4),
                  Block(n_embd, n_head=4),
                  nn.LayerNorm(n_embd),
            )
            self.lm_head = nn.Linear(n_embd, vocab_size) 
            
            
      def forward(self, idx, targets=None):
            B,T = idx.shape
            
            # idx and targets are both (B,T) tensor of integers
            tok_emb = self.token_embedding_table(idx) # (B,T,n_embd)
            pos_emb = self.position_embedding_table(torch.arange(T)) # (T,C)
            x = tok_emb + pos_emb
            # x = self.sa_heads(x)
            # x = self.ffwd(x) # (B,T,C)
            x = self.blocks(x)
            logits = self.lm_head(x) # (B,T,vocab_size)
            
            if targets is None:
                  loss = None
            else:
                  B,T,C = logits.shape
                  logits = logits.view(B*T, C)
                  targets = targets.view(B*T)
                  loss = F.cross_entropy(logits, targets)
            
            return logits, loss
      
      def generate(self, idx, max_new_tokens):
            # idx is (B,T) array of indices in the current context
            for _ in range(max_new_tokens):
                  # crop idx to the last block_size tokens
                  idx_cond = idx[:, -block_size: ]
                  # get the predictions
                  logits,loss = self(idx_cond)
                  # focus only on the last time step
                  logits = logits[:, -1,:] # becomes (B,C) # (B,T,C) ke T dim ke last C ko pick kra
                  # apply softmax to get probabilities
                  probs = F.softmax(logits, dim=1) # (B,C)
                  # sample from the distribution
                  idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
                  # append sampled index to the running sequence
                  idx = torch.cat((idx, idx_next), dim=1) # (B,T+1)
            return idx
      
      
      
      
      


In [20]:
# query vector = what am i looking for
# key vector = what do i contain


In [21]:
torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C)

head_size = 16
key = nn.Linear(C,head_size, bias= False)
query = nn.Linear(C,head_size, bias=False)
value = nn.Linear(C, head_size, bias= False)
k = key(x)    #(B,T,16)
q = query(x)  #(B,T,16)
wei = q @ k.transpose(-2,-1) * (head_size**-0.5)# (B,T,16) @ (B,16,T) --> (B,T,T)

tril = torch.tril(torch.ones(T,T))
# wei = torch.zeros((T,T))
wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei,dim=-1)

v= value(x)
out = wei @ v
# out = wei @ x

out.shape

torch.Size([4, 8, 16])